In [1]:
import os
import sys
import time
import datetime
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
import torch.nn as nn
from torchvision.utils import save_image, make_grid
import torchvision.transforms as transforms
import glob
import dlib
from pynput import keyboard
from pynput.mouse import Button, Controller

In [2]:
filepath = "/Users/gmmyung/Desktop/Develop/gazetrack" # indicate path of workspace, change the path suited to your Google drive setting
sys.path.append(filepath)

In [3]:
class eyeNet2(nn.Module):

    def __init__(self):
        super().__init__()
        self.right = nn.Sequential(nn.Conv2d(1, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3),
                                  nn.Conv2d(8, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3),
                                  nn.Conv2d(8, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3))
        self.left = nn.Sequential(nn.Conv2d(1, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3),
                                  nn.Conv2d(8, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3),
                                  nn.Conv2d(8, 8, kernel_size=3),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(inplace=True),
                                  nn.MaxPool2d(3))
        self.final = nn.Sequential(nn.Linear(200, 40),
                                   nn.ReLU(inplace=True),
                                   nn.Linear(40, 2),
                                   nn.ReLU(inplace=True))

    def forward(self, l, r, c):
        input_size = l.size()
        l, r = self.left(l), self.left(r)
        x = torch.cat((torch.flatten(l, start_dim=1), torch.flatten(r, start_dim=1), c), 1)
        x = self.final(x)
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

l = torch.zeros(16, 1, 100, 100)
r = torch.zeros(16, 1, 100, 100)
c = torch.zeros(16, 136)
model = eyeNet2()
start = time.time()
output = model(l, r, c)
print(time.time()-start)
print(output.size())

0.07892990112304688
torch.Size([16, 2])


/Users/gmmyung/miniconda3/envs/gazetrack/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
experiment = "exp_colab"
model.load_state_dict(torch.load(f"{filepath}/{experiment}/latest_model_state_dict.pt"))

<All keys matched successfully>

In [5]:
model.eval()
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
cropsize = 100

def on_press(key):
    if key == keyboard.Key.esc:
        print ('end pressed')
        breaker = True
        return False
    

listener_keyboard = keyboard.Listener(on_press=on_press)
listener_keyboard.start()
    
tran = transforms.ToTensor()  
breaker = False
mymouse = Controller()


print("Start logging!")
while 1:
    ret, frame = capture.read()
    img = frame
    rects = detector(img, 1)
    for i, rect in enumerate(rects):
        l = rect.left()
        t = rect.top()
        b = rect.bottom()
        r = rect.right()
        shape = predictor(img, rect)
        disp = img
        cordlist = []
        for j in range(68):
            p, q = shape.part(j).x, shape.part(j).y
            cordlist.append(p)
            cordlist.append(q)
            
        left_eye_x= sum([shape.part(j).x for j in [36, 37, 38, 39, 40, 41]]) // 6
        left_eye_y= sum([shape.part(j).y for j in [36, 37, 38, 39, 40, 41]]) // 6
        right_eye_x= sum([shape.part(j).x for j in [42, 43, 44, 45, 46, 47]]) // 6
        right_eye_y= sum([shape.part(j).y for j in [42, 43, 44, 45, 46, 47]]) // 6
        cropped_left = img[left_eye_y-cropsize//2: left_eye_y+cropsize//2, left_eye_x-cropsize//2: left_eye_x+cropsize//2]
        cropped_right = img[right_eye_y-cropsize//2: right_eye_y+cropsize//2, right_eye_x-cropsize//2: right_eye_x+cropsize//2]
        cropped_left = cv2.cvtColor(cropped_left, cv2.COLOR_BGR2GRAY)
        cropped_right = cv2.cvtColor(cropped_right, cv2.COLOR_BGR2GRAY)
        
        left_tensor = tran(cropped_left).float().unsqueeze(0)
        right_tensor = tran(cropped_right).float().unsqueeze(0)
        cord_tensor = torch.FloatTensor(cordlist).unsqueeze(0)
        print(cord_tensor.size())
        
        pred = model(left_tensor, right_tensor, cord_tensor)
        
        print(pred[0])
        mymouse.position = (float(pred[0][0]), float(pred[0][1]))
    print(breaker)
    if breaker == True:
        break
        
        

2022-02-13 23:24:27.373 python[77699:10104589] dalPluginMain


Start logging!
torch.Size([1, 136])
tensor([191.5575, 381.3261], grad_fn=<SelectBackward>)
False


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 136])
tensor([311.3606, 340.6750], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([363.8416, 330.1111], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([417.6698, 347.5809], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([438.1602, 392.5945], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([399.5111, 393.9977], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([381.7409, 392.5096], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([352.5267, 373.0706], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([329.9885, 366.7654], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([335.4286, 356.2375], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([356.2232, 362.1115], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([326.5609, 339.3601], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([335.5932, 338.0126], grad_fn=<SelectBackward>)
False
torch.Size([1, 1

torch.Size([1, 136])
tensor([435.0512, 358.5987], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([434.2780, 356.8962], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([435.8304, 359.8196], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([435.6526, 358.1106], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([436.7076, 351.8395], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([436.3470, 352.8997], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([436.1987, 355.4711], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([437.6012, 353.8119], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([419.1607, 356.8413], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([403.9350, 365.0098], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([377.2461, 377.1913], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([355.7600, 387.1924], grad_fn=<SelectBackward>)
False
torch.Size([1, 1

torch.Size([1, 136])
tensor([203.0656, 288.9397], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([245.0582, 323.6709], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([197.9377, 291.3245], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([196.7171, 284.7943], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([197.3149, 280.0776], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([196.9815, 279.4915], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([199.6494, 285.5355], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([199.5676, 281.4128], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([218.5067, 284.8067], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([256.3411, 287.1473], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([269.9840, 286.9185], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([275.5804, 287.2983], grad_fn=<SelectBackward>)
False
torch.Size([1, 1

torch.Size([1, 136])
tensor([255.9594, 367.0518], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([235.5164, 361.6789], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([236.8440, 363.5018], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([232.7967, 361.2990], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([225.8676, 361.7647], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([238.3897, 363.7676], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([231.6618, 356.1169], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([239.5833, 360.3271], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([249.3306, 372.4466], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([246.4914, 368.4390], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([238.5259, 358.1086], grad_fn=<SelectBackward>)
False
torch.Size([1, 136])
tensor([239.8620, 360.2948], grad_fn=<SelectBackward>)
False
torch.Size([1, 1

KeyboardInterrupt: 